In [2]:
!pip install google-cloud-aiplatform>=1.16.0
!pip install kfp>=1.8.0
!pip install google-cloud-storage>=1.44.0
!pip install google-cloud-speech>=2.0.0


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
pip install ffmpeg


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import json
import tempfile
import subprocess
from typing import NamedTuple

from kfp.v2 import dsl
from kfp.v2 import compiler
from google.cloud import aiplatform
from google.cloud import storage
from google.cloud import speech_v1p1beta1 as speech

/var/tmp/ipykernel_8977/2029902502.py:7: DeprecationWarning: The module `kfp.v2` is deprecated and will be removed in a futureversion. Please import directly from the `kfp` namespace, instead of `kfp.v2`.
  from kfp.v2 import dsl


In [5]:
PROJECT_ID = "resolute-winter-447814-t5"
LOCATION = "us-central1"
INPUT_BUCKET = "resolute-winter-447814-t5_input"
OUTPUT_BUCKET = "resolute-winter-447814-t5_output"
SERVICE_ACCOUNT = "232486347340-compute@developer.gserviceaccount.com"
INPUT_VIDEO = "gs://resolute-winter-447814-t5_input/videos/Introducing Yourself - Phrases ( lingoneo.org ).mp4"


In [24]:
@dsl.component(
    base_image="python:3.9",
    packages_to_install=["ffmpeg-python", "google-cloud-storage"],
)
def extract_audio_from_video(
    input_video_gcs_path: str,
    output_audio_gcs_path: str
) -> str:
    """Extract MP3 audio from MP4 video."""
    import os
    import subprocess
    import tempfile
    from google.cloud import storage
    
    print(f"Extracting audio from {input_video_gcs_path} to {output_audio_gcs_path}")
    
    # Install FFmpeg directly in the component
    print("Installing FFmpeg...")
    subprocess.run(["apt-get", "update", "-y"], check=True)
    subprocess.run(["apt-get", "install", "-y", "ffmpeg"], check=True)
    print("FFmpeg installed successfully")
    
    # Create temporary directory for processing
    with tempfile.TemporaryDirectory() as temp_dir:
        # Download video from GCS
        storage_client = storage.Client()
        
        # Parse bucket and blob names
        input_path = input_video_gcs_path.replace("gs://", "")
        bucket_name = input_path.split("/")[0]
        blob_name = "/".join(input_path.split("/")[1:])
        
        # Get bucket and blob
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        
        # Download to temporary file
        local_video_path = os.path.join(temp_dir, os.path.basename(blob_name))
        blob.download_to_filename(local_video_path)
        print(f"Video downloaded to {local_video_path}")
        
        # Extract audio using FFmpeg
        local_audio_path = os.path.join(temp_dir, os.path.splitext(os.path.basename(blob_name))[0] + ".mp3")
        print(f"Extracting audio to {local_audio_path}")
        
        # Using subprocess for FFmpeg
        cmd = [
            "ffmpeg", "-i", local_video_path, 
            "-vn",  # No video
            "-acodec", "mp3",  # MP3 codec
            "-ab", "192k",  # Bitrate
            "-ar", "44100",  # Sample rate
            "-y",  # Overwrite output file
            local_audio_path
        ]
        
        try:
            subprocess.run(cmd, check=True, capture_output=True)
            print("Audio extraction completed successfully")
        except subprocess.CalledProcessError as e:
            print(f"FFmpeg error: {e.stderr.decode()}")
            raise RuntimeError(f"Failed to extract audio: {e}")
        
        # Upload extracted audio to GCS
        print(f"Uploading audio to {output_audio_gcs_path}")
        output_path = output_audio_gcs_path.replace("gs://", "")
        output_bucket_name = output_path.split("/")[0]
        output_blob_name = "/".join(output_path.split("/")[1:])
        
        output_bucket = storage_client.bucket(output_bucket_name)
        output_blob = output_bucket.blob(output_blob_name)
        output_blob.upload_from_filename(local_audio_path)
        
        print(f"Audio extraction and upload complete")
        
        return output_audio_gcs_path


In [56]:
from kfp import dsl
from kfp.dsl import component

@component(
    base_image="python:3.9",
    packages_to_install=["assemblyai", "google-cloud-storage"],
)
def transcribe_audio_assemblyai(
    audio_gcs_path: str,
    output_transcript_gcs_path: str,
    assemblyai_api_key: str,
    language_code: str = "en",
     model: str = "chirp" # New model parameter with default "chirp"
) -> str:
    """Transcribes audio from Google Cloud Storage using AssemblyAI's Chirp model."""

    import os
    import json
    import assemblyai as aai
    from google.cloud import storage
    import tempfile

    print(f"Transcribing audio from {audio_gcs_path} using AssemblyAI Chirp to {output_transcript_gcs_path}")

    try:
        # Initialize GCS client
        storage_client = storage.Client()

        # Create a temporary directory
        with tempfile.TemporaryDirectory() as temp_dir:
            # Construct local audio path
            local_audio_path = os.path.join(temp_dir, "audio.mp3")

            # Download the audio from GCS
            print(f"Downloading {audio_gcs_path} to {local_audio_path}")
            #---------GCS VALIDATION------
            input_path = audio_gcs_path.replace("gs://", "")
            if not input_path:
                raise ValueError("Invalid audio_gcs_path: cannot be empty.")

            input_parts = input_path.split("/")

            if len(input_parts) < 2:
                raise ValueError(
                    "Invalid audio_gcs_path: must be in the format "
                    "gs://bucket-name/path/to/audio.mp3"
                )

            bucket_name = input_parts[0]
            blob_name = "/".join(input_parts[1:])


            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)
            blob.download_to_filename(local_audio_path) #Download the file
            print(f"Audio downloaded successfully to {local_audio_path}")
            #---------------------------

            # Configure AssemblyAI
            aai.settings.api_key = assemblyai_api_key
            transcriber = aai.Transcriber()
            transcriber.model = model #Setting transcribe Object
            # Configuration for AssemblyAI
            config = aai.TranscriptionConfig(
                speaker_labels=True,  # Enable speaker diarization
                language_code=language_code # Specify the language
            )

            # Transcribe the *local* audio file
            print(f"Transcribing local file {local_audio_path}")
            transcript = transcriber.transcribe(local_audio_path, config)

            if transcript:
                print("Transcript Status:", transcript.status)
                transcript_data = {
                    "text": transcript.text,
                    "status": transcript.status,
                    "words": []  #Add the word-level info if needed in the transcript_data
                }

                #If you need word-level information (times, confidences):
                if transcript.words:
                    for word in transcript.words:
                        word_data = {
                            "text": word.text,
                            "confidence": word.confidence,
                            "speaker": word.speaker
                        }
                        if hasattr(word, "start"):
                            word_data["start"] = word.start
                        else:
                            word_data["start"] = None
                        if hasattr(word, "end"):
                            word_data["end"] = word.end
                        else:
                            word_data["end"] = None

                        transcript_data["words"].append(word_data)

                # Upload transcript to GCS
                # Ensure storage_client is initialized only once
                with tempfile.TemporaryDirectory() as temp_dir2:
                    local_transcript_path = os.path.join(temp_dir2, "transcript.json")

                    with open(local_transcript_path, "w") as f:
                        json.dump(transcript_data, f, indent=2)

                    # Upload to GCS
                    print(f"Uploading transcript to {output_transcript_gcs_path}")

                    output_path = output_transcript_gcs_path.replace("gs://", "")

                    if not output_path:
                        raise ValueError("Invalid output_transcript_gcs_path: cannot be empty.")

                    parts = output_path.split("/")

                    if len(parts) < 2:
                        raise ValueError(
                            "Invalid output_transcript_gcs_path: must be in the format "
                            "gs://bucket-name/path/to/transcript.json"
                        )

                    output_bucket_name = parts[0]
                    output_blob_name = "/".join(parts[1:])

                    output_bucket = storage_client.bucket(output_bucket_name)
                    output_blob = output_bucket.blob(output_blob_name)

                    print(f"Attempting to upload {local_transcript_path} to {output_transcript_gcs_path}")
                    output_blob.upload_from_filename(local_transcript_path)
                    print(f"Upload completed successfully")


                print(f"Transcription saved to {output_transcript_gcs_path}")

                return output_transcript_gcs_path

            else:
                print("Transcription failed.")
                raise ValueError("AssemblyAI transcription failed.")

    except Exception as e:
        print(f"An error occurred: {e}")
        raise e # Re-raise the exception for KFP to handle

In [53]:
@dsl.component(
    base_image="python:3.9",
    packages_to_install=["google-cloud-storage"],
)
def generate_subtitles(
    transcript_gcs_path: str,
    output_subtitles_gcs_path: str,
    max_chars_per_line: int = 42
) -> str:
    """Generate an SRT file that formats words into sentences while preserving exact timing."""
    
    import os
    import json
    import tempfile
    import re
    from google.cloud import storage
    from collections import defaultdict

    print(f"Generating sentence-based subtitles from {transcript_gcs_path} to {output_subtitles_gcs_path}")

    storage_client = storage.Client()
    
    try:
        # Download transcript from GCS
        input_path = transcript_gcs_path.replace("gs://", "")
        bucket_name = input_path.split("/")[0]
        blob_name = "/".join(input_path.split("/")[1:])
        
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)
        
        with tempfile.TemporaryDirectory() as temp_dir:
            local_transcript_path = os.path.join(temp_dir, "transcript.json")
            try:
                blob.download_to_filename(local_transcript_path)
            except Exception as e:
                print(f"Error downloading transcript: {e}")
                raise
            
            # Load transcript
            try:
                with open(local_transcript_path, "r") as f:
                    transcript_data = json.load(f)
            except Exception as e:
                print(f"Error loading transcript JSON: {e}")
                raise
                
            words = transcript_data.get("words", [])
            print(f"Number of words in transcript: {len(words)}")  # Check number of words
            if not words:
                print("No word-level timing information found in transcript")
                return None

            def format_time(milliseconds):
                """Convert milliseconds to SRT time format (HH:MM:SS,mmm)"""
                seconds = float(milliseconds) / 1000  # Convert to seconds
                hours = int(seconds // 3600)
                minutes = int((seconds % 3600) // 60)
                seconds = seconds % 60
                return f"{hours:02d}:{minutes:02d}:{seconds:06.3f}".replace(".", ",")
        
            # Function to group words into sentences based on punctuation
            def is_sentence_end(word):
                """Check if a word marks the end of a sentence."""
                return bool(re.search(r'[.!?]$', word))
            
            srt_content = ""
            subtitle_count = 1
            i = 0
            error_count = defaultdict(int) # For Logging statistics
            while i < len(words):
                # Use the correct key name "start" and "end" and require text
                if "start" not in words[i] or "end" not in words[i] or "text" not in words[i]:
                    print(f"Skipping word at index {i} due to missing timing/text information.")
                    error_count["missing"] += 1 # Keep track of the issue
                    i += 1
                    continue

                start_time = words[i]["start"]
                end_time = words[i]["end"]
                
                sentence_words = []
                
                while i < len(words):
                    word = words[i]
                    if "start" not in words[i] or "end" not in words[i] or "text" not in words[i]:
                        print(f"Skipping word at index {i} due to missing timing/text information inside inner loop.")
                        error_count["inner_missing"] += 1  # Track errors in the inner loop
                        i += 1
                        break

                    sentence_words.append(word["text"]) #Use text
                    end_time = word["end"]  # Use "end"

                    if is_sentence_end(word["text"]) or i == len(words) - 1:  # use "text"
                        break
                    
                    i += 1

                i += 1  # Move to the next word for the next subtitle

                formatted_lines = []
                current_line = []
                current_line_chars = 0

                for word in sentence_words:
                    if current_line_chars + len(word) + 1 > max_chars_per_line and current_line:
                        formatted_lines.append(" ".join(current_line))
                        current_line = [word]
                        current_line_chars = len(word)
                    else:
                        current_line.append(word)
                        current_line_chars += len(word) + (1 if current_line_chars > 0 else 0)

                if current_line:
                    formatted_lines.append(" ".join(current_line))

                srt_content += f"{subtitle_count}\n"
                srt_content += f"{format_time(start_time)} --> {format_time(end_time)}\n"
                srt_content += "\n".join(formatted_lines) + "\n\n"
                subtitle_count += 1

            print(f"Final SRT Statistics: {error_count}") # Report
            local_srt_path = os.path.join(temp_dir, "subtitles.srt")
            with open(local_srt_path, "w") as f:
                f.write(srt_content)

            output_path = output_subtitles_gcs_path.replace("gs://", "")
            output_bucket_name = output_path.split("/")[0]
            output_blob_name = "/".join(output_path.split("/")[1:])

            output_bucket = storage_client.bucket(output_bucket_name)
            output_blob = output_bucket.blob(output_blob_name)
            output_blob.upload_from_filename(local_srt_path)

            print(f"Generated precise sentence-based subtitles and uploaded to {output_subtitles_gcs_path}")

            return output_subtitles_gcs_path

    except Exception as e:
        print(f"An error occurred: {e}")
        raise

In [57]:
@dsl.component(
    base_image="python:3.9",
    packages_to_install=["ffmpeg-python", "google-cloud-storage"],
)
def overlay_subtitles_on_video(
    input_video_gcs_path: str,
    subtitles_gcs_path: str,
    output_video_gcs_path: str,
    font_size: int = 48,
    font_color: str = "white"
) -> str:
    """Overlay SRT subtitles on video ensuring exact sentence-based sync."""
    
    import os
    import subprocess
    import tempfile
    from google.cloud import storage

    print(f"Overlaying subtitles from {subtitles_gcs_path} on {input_video_gcs_path}")

    # Install FFmpeg
    subprocess.run(["apt-get", "update", "-y"], check=True)
    subprocess.run(["apt-get", "install", "-y", "ffmpeg"], check=True)

    storage_client = storage.Client()
    
    with tempfile.TemporaryDirectory() as temp_dir:
        # Download video from GCS
        input_video_path = input_video_gcs_path.replace("gs://", "")
        video_bucket_name = input_video_path.split("/")[0]
        video_blob_name = "/".join(input_video_path.split("/")[1:])
        
        video_bucket = storage_client.bucket(video_bucket_name)
        video_blob = video_bucket.blob(video_blob_name)
        
        local_video_path = os.path.join(temp_dir, os.path.basename(video_blob_name))
        video_blob.download_to_filename(local_video_path)

        # Download subtitles from GCS
        subtitles_path = subtitles_gcs_path.replace("gs://", "")
        subtitles_bucket_name = subtitles_path.split("/")[0]
        subtitles_blob_name = "/".join(subtitles_path.split("/")[1:])
        
        subtitles_bucket = storage_client.bucket(subtitles_bucket_name)
        subtitles_blob = subtitles_bucket.blob(subtitles_blob_name)
        
        local_subtitles_path = os.path.join(temp_dir, "subtitles.srt")
        subtitles_blob.download_to_filename(local_subtitles_path)

        # Output video path
        local_output_video_path = os.path.join(
            temp_dir, 
            f"{os.path.splitext(os.path.basename(video_blob_name))[0]}_with_subtitles.mp4"
        )

        # Overlay subtitles using FFmpeg with exact timing
        print("Overlaying subtitles with sentence-based exact sync")
        cmd = [
            "ffmpeg", "-i", local_video_path,
            "-vf", f"subtitles={local_subtitles_path}",
            "-c:a", "copy",  # Preserve original audio
            "-y",  # Overwrite output file
            local_output_video_path
        ]
        
        try:
            subprocess.run(cmd, check=True, capture_output=True)
            print("Subtitle overlay completed successfully with exact sentence sync")
        except subprocess.CalledProcessError as e:
            print(f"FFmpeg error: {e.stderr.decode()}")
            raise RuntimeError("Failed to overlay subtitles with exact sync")

        # Upload output video to GCS
        output_path = output_video_gcs_path.replace("gs://", "")
        output_bucket_name = output_path.split("/")[0]
        output_blob_name = "/".join(output_path.split("/")[1:])
        
        output_bucket = storage_client.bucket(output_bucket_name)
        output_blob = output_bucket.blob(output_blob_name)
        output_blob.upload_from_filename(local_output_video_path)

        print(f"Video with exact subtitles uploaded to {output_video_gcs_path}")
        
        return output_video_gcs_path


In [58]:
from typing import NamedTuple

@dsl.pipeline(
    name="video-processing-pipeline",
    description="A pipeline that processes MP4 videos, extracts audio, generates transcriptions, and overlays subtitles"
)
def video_processing_pipeline(
    input_video_gcs_path: str,
    output_bucket: str,
    assemblyai_api_key: str,
    language_code: str = "en",
    model: str = "chirp"
) -> NamedTuple('Outputs', [('output_video', str)]):  # Add this return type annotation
    """Pipeline that processes video files with subtitle generation."""
    
    # Define output paths
    video_basename = "video"  # Using fixed names to avoid string manipulation in pipeline
    
    output_audio_gcs_path = f"gs://{output_bucket}/output/audio/{video_basename}.mp3"
    output_transcript_gcs_path = f"gs://{output_bucket}/output/transcripts/{video_basename}.json"
    output_subtitles_gcs_path = f"gs://{output_bucket}/output/subtitles/{video_basename}.srt"
    output_video_gcs_path = f"gs://{output_bucket}/output/videos/{video_basename}_with_subtitles.mp4"
    
    # Step 1: Extract audio from video
    extract_task = extract_audio_from_video(
        input_video_gcs_path=input_video_gcs_path,
        output_audio_gcs_path=output_audio_gcs_path
    )
    
    # Step 2: Transcribe audio to text
    transcribe_task = transcribe_audio_assemblyai(
        audio_gcs_path=extract_task.output,
        assemblyai_api_key=assemblyai_api_key,
        output_transcript_gcs_path=output_transcript_gcs_path,
        language_code=language_code,
        model=model
    )
    
    # Step 3: Generate subtitles from transcription
    subtitles_task = generate_subtitles(
        transcript_gcs_path=transcribe_task.output,
        output_subtitles_gcs_path=output_subtitles_gcs_path
    )
    
    # Step 4: Overlay subtitles on video
    overlay_task = overlay_subtitles_on_video(
        input_video_gcs_path=input_video_gcs_path,
        subtitles_gcs_path=subtitles_task.output,
        output_video_gcs_path=output_video_gcs_path
    )
    
    # Return the final output with proper naming
    return NamedTuple('Outputs', [('output_video', str)])(overlay_task.output)

# STEP 6: Compile the Pipeline
compiler.Compiler().compile(
    pipeline_func=video_processing_pipeline,
    package_path="video_processing_pipeline.json"
)

print("Pipeline compiled successfully to video_processing_pipeline.json")


Pipeline compiled successfully to video_processing_pipeline.json
